In [20]:
# necessary
import numpy as np
import pandas as pd
from tqdm.auto import tqdm

# db connection 
import pymysql
from sqlalchemy import create_engine


class AccessDataBase():
    
    def __init__(self, user_name, password, db_name):
        self.user_name = user_name
        self.password = password
        self.db_name = db_name
    
    def db_connect(self):

        ''' db connect '''

        host_url = "db.ds.mycelebs.com"
        port_num = 3306
        conn = pymysql.connect(host=host_url, user=self.user_name, passwd=self.password, port=port_num, db=self.db_name, charset='utf8')
        curs = conn.cursor(pymysql.cursors.DictCursor)
        return curs

    
    def get_tbl(self, table_name, columns):
        
        ''' db에서 원하는 테이블, 컬럼 pd.DataFrame에 할당 '''
        
        curs = self.db_connect()
        
        if columns == 'all':
            sql_query = f'SELECT * FROM {table_name};'
            
        
        else:
            # SELECT columns
            sql_query = 'SELECT '
            i = 0
            for col in columns:
                if i == 0:
                    sql_query += col

                else:
                    sql_query += ', ' + col

                i += 1

            # FROM table_name
            sql_query += f' FROM {table_name};'
        
        curs.execute(sql_query)
        tbl = curs.fetchall()
        df = pd.DataFrame(tbl)
        curs.close()
        
        return df

    def engine_upload(self, upload_df, table_name, if_exists_option):
        ''' Create Table '''
        
        host_url = "db.ds.mycelebs.com"
        port_num = 3306
        engine = create_engine(f'mysql+pymysql://{self.user_name}:{self.password}@{host_url}:{port_num}/{self.db_name}?charset=utf8mb4')
        engine_conn = engine.connect()
        upload_df.to_sql(table_name, engine_conn, if_exists=if_exists_option, index=None)
        engine_conn.close()
        engine.dispose()
        
        

In [21]:
un = 'yeonseosla'
pw = 'jys9807'
sm = 'beauty_kr'

db = AccessDataBase(un, pw, sm)

In [22]:
# df = pd.DataFrame([[0,1], [1,1], [2,0]], columns=['id', 'status'])
table_name = 'test_table'
db.engine_upload(df, table_name, "append")

In [16]:
# df.loc[:, 'string'] = str(["a", "b", "c"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   id      3 non-null      int64         
 1   status  3 non-null      int64         
 2   date    3 non-null      datetime64[ns]
 3   string  3 non-null      object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 224.0+ bytes


In [3]:
def get_tbl_name():
    ''' db에 존재하는 모든 테이블 이름 가져오기 '''

    curs = db.db_connect()

    # get table name list
    query = "SHOW TABLES;"
    curs.execute(query)
    tables = curs.fetchall()

    table_list = []
    for table in tables:
        tbl = list(table.values())[0]
        table_list.append(tbl)

    return table_list

In [4]:
def get_tbl_columns(table_name):
    ''' 선택한 테이블 컬럼 가져오기 '''

    curs = db.db_connect()

    # get table columns 
    query = f"SHOW FULL COLUMNS FROM {table_name};"
    curs.execute(query)
    columns = curs.fetchall()

    column_list = []
    for column in columns:
        field = column['Field']
        column_list.append(field)

    return column_list

In [66]:
import re
tables = get_tbl_name()
reg = re.compile('naver_beauty_product_info_extended_v[0-9]+')
table_list = []
for tbl in tables:
    tbl_ = re.match(reg, tbl)
    if tbl_:
        table_list.append(tbl_.group(0))
        
table_list = sorted(list(set(table_list)))
    